### Задание 1. Обязательная часть

Из страницу с новостями на habr.com/ru/all/ нужны статьи, в которых есть ключевые слова KEYWORDS. 

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

*В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [211]:
import requests
import time
import pandas as pd

In [10]:
from bs4 import BeautifulSoup

In [26]:
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг']

In [59]:
news = requests.get(URL)
soup = BeautifulSoup(news.text, 'html.parser')
#soup

In [16]:
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/aio350/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/076/1e1/2c6/0761e12c6ea091cb6170c185394fae64.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">aio350</span>
 </a>
 <span class="post__time">сегодня в 13:15</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/515250/">Краткое руководство по Node.js для начинающих (SPA, PWA, mobile-first)</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/webdev/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Разработка веб-сайтов'); }" rel="nofollow" title="Вы не подписан

In [70]:
for post in posts:
    
    hubs = post.find_all('a', class_='hub-link')
#    print(post)
    for hub in hubs:
            hub_lower = hub.text.lower()
    
            if any([key in hub_lower for key in KEYWORDS]):
                title_element = post.find('a', class_='post__title_link')
                date_element = post.find('span', class_='post__time').text
                print(date_element, title_element.text, title_element.attrs.get('href'))

вчера в 21:56 Визуализация статистики Яндекс Директ своими руками. От API до Data Studio https://habr.com/ru/post/515222/


In [50]:
# дополнительное задание
def get_all_links(url):
    all_refs = []
    
    res = requests.get(URL)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')

    articles = soup.find_all('article', class_='post')
    all_refs = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), articles))

    return all_refs

all_links = get_all_links(URL)
all_links[:3]

['https://habr.com/ru/post/515256/',
 'https://habr.com/ru/post/515252/',
 'https://habr.com/ru/post/515250/']

In [73]:
kom_news = pd.DataFrame()
for link in all_links:

    soup_new = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    soup_new_low = soup_new.text.lower()

    if any([key in soup_new_low for key in KEYWORDS]):
        
        date = pd.to_datetime(soup_new.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
        title = soup_new.find('title').text
        row = {'date': date, 'title': title, 'link': link}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])]) 
kom_news

,date,title,link
0,2020-08-15,Как нанять 50 синьоров за 43 дня и быстро вклю...,https://habr.com/ru/company/ruvds/blog/515182/
0,2020-08-15,Чтение книг на английском с помощью сервиса JR...,https://habr.com/ru/post/515240/
0,2020-08-15,"Что происходит, когда вы обновляете свой DNS /...",https://habr.com/ru/company/mailru/blog/513844/
0,2020-08-14,Квадратичное финансирование / Хабр,https://habr.com/ru/post/515212/
0,2020-08-14,Визуализация статистики Яндекс Директ своими р...,https://habr.com/ru/post/515222/


### Задание 2. Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 

*Датафрейм должен иметь столбцы: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [142]:
import json

In [193]:
EMAIL = ['ada@severtm.ru', 'zobr2005@mail.ru', 'd.anchunov@gmail.com']

In [210]:
def checking_your_email(e_mail):
    URL = 'https://digibody.avast.com/v1/web/leaks'
    data = {
        'email': e_mail
    }
    req = requests.post(URL, json=data)
    your_passwords_have_leaked = pd.DataFrame()
    
    for q in req.json()['value']:
        
        date = time.ctime(q['leak_date']/1000)
        title = q['leak_info']['title']
        info = q['leak_info']['description']

        row = {'email': e_mail, 'date': date, 'title': title, 'info':info}
        your_passwords_have_leaked = your_passwords_have_leaked.append([row])

    return your_passwords_have_leaked

In [209]:
test_result = pd.DataFrame()
for email in EMAIL:
    test_result = test_result.append(checking_your_email(email))
    
test_result.head()

,email,date,title,info
0,zobr2005@mail.ru,Wed Feb 6 03:00:00 2019,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
0,zobr2005@mail.ru,Fri Oct 21 03:00:00 2016,Adobe Systems,"In October of 2013, criminals penetrated Adobe..."
0,zobr2005@mail.ru,Tue Jan 29 03:00:00 2019,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
0,zobr2005@mail.ru,Fri Dec 13 03:00:00 2019,Sensitive Source,This source has been marked as sensitive due t...
0,d.anchunov@gmail.com,Wed Mar 20 03:00:00 2019,Bookmate,"In July 2018, Bookmate was allegedly breached...."
